# Notebook Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path += ["../"]

In [3]:
import warnings
warnings.filterwarnings("ignore")

Note that we are using the local ds_nudge_up repo only for loading the utils.
The rest is done using the pip installed library.

In [4]:
import os
import src.mosaiks.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

In [5]:
os.getcwd()

'/home/jovyan/mosaiks/playground'

In [6]:
!pip install -e ..

Obtaining file:///home/jovyan/mosaiks
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mosaiks (pyproject.toml) ... done
  Created wheel for mosaiks: filename=mosaiks-0.0.1rc0-0.editable-py3-none-any.whl size=1786 sha256=572209c6773ac7935ccdbe82641b55c35fea9dea1be63df61402ce579867ccae
  Stored in directory: /tmp/pip-ephem-wheel-cache-mvzmt7ps/wheels/8c/b8/09/ac86a95e312dc7e48173fa88b434bf7660e68925ca07d32c2c
Successfully built mosaiks


In [7]:
# !pip uninstall mosaiks -y

🚨🚨 **Make sure you update github token in the secrets file** 🚨🚨 

In [8]:
# secrets = utl.load_yaml_config("../config/secrets.yml")
# GITHUB_TOKEN = secrets["GITHUB_TOKEN"]
# mosaiks_package_link = f"git+https://{GITHUB_TOKEN}@github.com/IDinsight/ds_nudge_up@as-package"

In [9]:
# !pip install {mosaiks_package_link} --upgrade

# Setup Dask Cluster and Client

## Local Cluster

4 workers with 4 threads each seem to work best. A lot of time a thread is waiting on data to load so CPU is underutilized.

In [6]:
import logging
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=4, processes=True, threads_per_worker=4, silence_logs=logging.ERROR)
client = Client(cluster)
client

2023-03-03 09:46:59,596 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/mosaiks/playground/dask-worker-space/worker-8zk5doj7', purging
2023-03-03 09:46:59,596 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/mosaiks/playground/dask-worker-space/worker-3ipq485y', purging
2023-03-03 09:46:59,597 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/mosaiks/playground/dask-worker-space/worker-o7i1747e', purging
2023-03-03 09:46:59,597 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/mosaiks/playground/dask-worker-space/worker-vwzq3wn2', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/amirali1376@gmail.com/proxy/8787/status,
Dashboard: /user/amirali1376@gmail.com/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 27.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43545,Workers: 4
Dashboard: /user/amirali1376@gmail.com/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 27.00 GiB
Comm: tcp://127.0.0.1:40373,Total threads: 4
Dashboard: /user/amirali1376@gmail.com/proxy/44281/status,Memory: 6.75 GiB
Nanny: tcp://127.0.0.1:36253,


## Gateway cluster

In [11]:
# from dask_gateway import Gateway
# import dask_gateway
# from dask.distributed import PipInstall

# gateway = Gateway()
# options = gateway.cluster_options()
# options

In [12]:
# from dask.distributed import PipInstall

# cluster = gateway.new_cluster(options)
# client = cluster.get_client()
# print(cluster.dashboard_link)

# plugin = PipInstall(packages=[mosaiks_package_link], pip_options=["--upgrade"], restart=False)
# client.register_worker_plugin(plugin)

# cluster.scale(10)

In [13]:
# cluster.shutdown()

# Load params

In [7]:
from mosaiks.featurize import *

from dask import delayed
from dask.distributed import as_completed
from time import sleep
import pandas as pd
import numpy as np

In [8]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_search_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

# Load point coords

In [9]:
centroid_points_gdf = utl.load_points_gdf(**data_sources['request_points_centroids'])

In [10]:
# only keep points in the focus states
focus_states_id_dict = {
    20:"jharkhand",
    22:"chhattisgarh",
    8:"rajasthan",
    23:"madhya pradesh",
    18:"assam",
    16:"tripura",
}

focus_states_filter = centroid_points_gdf["pc11_s_id"].isin(focus_states_id_dict.keys())
points_gdf_focus = centroid_points_gdf[focus_states_filter]
points_gdf_focus.shape

(179716, 9)

In [11]:
points_gdf = points_gdf_focus  #.sample(200, random_state=0) # Select random 200 points (for testing)

# Fetch image stac refs

`fetch_image_refs` now returns a dask dataframe and is not yet computed. So it finishes quite quickly.

🌱 **SUGGESTION:** Change `n_partitions` parameter to `n_per_partition` and calculate `n_partitions` here? This will ensure each partition takes a similar amount of time to process even across differing point-set sizes.

In [23]:
%%time
points_gdf_with_stac = fetch_image_refs(
    points_gdf, 
    featurization_params['dask']['n_partitions'],
    featurization_params['satellite_search_params']
)

CPU times: user 2.24 s, sys: 151 ms, total: 2.39 s
Wall time: 8.21 s


# Define delayed objects

We use the `delayed` decorator to turn our function into a delayed function. This means it will not run immediately when called but instead return a delayed object that can be run later

In [24]:
@delayed
def partition_run(df, satellite_config, featurization_params, model, device):
    
    data_loader = create_data_loader(df, satellite_config, featurization_params['batch_size'])
    X_features = create_features(data_loader, featurization_params['num_features'], len(df), 
                             model, device, satellite_config['min_image_edge'])
    
    df = pd.DataFrame(X_features, index=df.index.copy())
    
    return df

We want to convert our dask dataframe into "delayed" objects. Each partition is now a delayed pandas dataframe and can be passed to our delayed function above

In [25]:
partitions = points_gdf_with_stac.to_delayed()

In [26]:
model = RCF(featurization_params['num_features'], 
            featurization_params['kernel_size'], 
            len(satellite_config['bands']))

Batch size of 10 seems to be optimal balance between maximally using the CPU and not blowing up the memory

In [27]:
featurization_params['batch_size']

10

# Run in parallel

## Trial run

The cell below will only run it for 8 of the partitions. That seems to be about how many we can do in parallel on a local cluster. We may be able to do more on a Gateway Cluster once that is working.

There are also better schemes. For example, kick off another partitions whenever one finishes. That might be a better use of resources.

In [ ]:
%%time

i = 0
p = partitions[i]
f = partition_run(p, satellite_config, featurization_params, model, 'cuda', dask_key_name=f'run_{i}')
df_future = client.compute(f)
for f in as_completed([df_future]):
    df = f.result()

In [ ]:
df

In [ ]:
print("Average feature value:", df.mean().mean())
df.iloc[0].hist()
_ = client.restart()

With 100k points and 200 partitions, 8 partitions should take ~7-8 minutes on an MPC GPU instance. So that's <1 minute per partition. If nothing goes wrong, the whole job should finish in <4 hours.

In theory, objects should get garbage collected once there are no references to them. But it seems to take forever (or never!) for python to do that. Possibly since we have a lot of nested things and a model object that we are still holding a reference to.

Restarting the cluster seems to be the sure way of clearing worker memory.

In [ ]:
_ = client.restart()

## Full run

This is going to create 200 dataframes - one for each partition. If any fail, we can always just rerun that single component.

In [ ]:
from datetime import datetime

In [ ]:
N_PARTITIONS = len(partitions)
N_PER_RUN = 8
START_IDX = 0
str_column_names = [str(i) for i in range(featurization_params['num_features'])]

folder_name = "2021_l8_centroids"
path_to_folder = "../data/01_preprocessed/mosaiks_features/"+folder_name
if not os.path.exists(path_to_folder):
    os.makedirs(path_to_folder)

In [ ]:
p_ids = np.arange(START_IDX, N_PARTITIONS + N_PER_RUN, N_PER_RUN)

failed_list = []
for p_start_id, p_end_id in zip(p_ids[:-1], p_ids[1:]):
    now = datetime.now().strftime("%d-%b %H:%M:%S")
    print(f"{now} Running batch: ", p_start_id, "to", p_end_id-1)
    
    delayed_dfs = []
    for i, p in enumerate(partitions[p_start_id:p_end_id]):
        f = partition_run(
            p, 
            satellite_config, 
            featurization_params, 
            model, 
            featurization_params['device'], 
            dask_key_name=f'features_{p_start_id + i}'
        )
        delayed_dfs.append(f)
    futures_dfs = client.compute(delayed_dfs)
    
    for f in as_completed(futures_dfs):
        try:
            df = f.result()
            df.columns = str_column_names
            df.to_parquet(f'{path_to_folder}/df_{f.key}.parquet.gzip', compression='gzip')

        except Exception as e:
            f_key = f.key
            partition_id = int(f_key.split("features_")[1])
            print(f"Partition {partition_id} failed. Error:", e)
            failed_list.append(partition_id)
        
    client.restart()
    sleep(5)

In [ ]:
# client.shutdown()

## Re-run failed partitions

Use this to just run partitions that failed

In [ ]:
%%time

FAILED_IDX = failed_list #[44]

delayed_dfs = []
failed_list_1 = []
for i in FAILED_IDX:
    p = partitions[i]
    f = partition_run(
        p, 
        satellite_config, 
        featurization_params, model, 
        featurization_params['device'], 
        dask_key_name=f'features_{i}'
    )
    delayed_dfs.append(f)
    futures_dfs = client.compute(delayed_dfs)
    
    for f in as_completed(futures_dfs):
        try:
            df = f.result()
            df.columns = str_column_names
            df.to_parquet(f'{path_to_folder}/df_{f.key}.parquet.gzip', compression='gzip')
        except Exception as e:
            print(f"Partition {f.key} failed. Error:", e)
            failed_list_1.append(f.key)

In [ ]:
# _ = client.restart()

# Load checkpoint files and combine

In [ ]:
all_files = os.listdir(path_to_folder)
parquet_files = sorted([file for file in all_files if file.endswith('.gzip')])
pd.Series(parquet_files).to_csv(f"{path_to_folder}/file_list.csv")
# parquet_files

In [ ]:
dfs = []
for filename in parquet_files:
    df = pd.read_parquet(path_to_folder+"/"+filename)
    dfs.append(df)

combined_df = pd.concat(dfs, axis=0)
combined_df.sort_index(inplace=True)

# add LatLons
combined_df = combined_df.join(points_gdf[["Lat","Lon"]])

print("Dataset size in memory (MB):", combined_df.memory_usage().sum() / 1000000)

In [ ]:
combined_df.to_parquet(path_to_folder+"/combined.parquet.gzip", compression="gzip")

In [ ]:
# combined_df_deduped = combined_df.drop_duplicates(subset=["Lat", "Lon", "0"])

In [ ]:
# df = pd.read_parquet(path_to_folder+"/combined.parquet.gzip")
# df